In [19]:
import numpy as np
import pandas as pd
import os
import xlrd
import xlwt
import re

In [20]:
excel_files = [f for f in os.listdir("data/")]

In [21]:
file = 'RDO No. 105 - Surigao City, Surigao del Norte.xls'
test = pd.ExcelFile(f"data/{file}")

In [22]:
sheet_names = sorted([name for name in test.sheet_names if name.strip().lower().startswith('sheet')], key=lambda name: int(re.search(r'\d+', name).group()))
last_sheet_name = sheet_names[-1] if sheet_names else None

In [23]:
last_sheet_name

'Sheet 5 (DO 034-2023)'

In [24]:
df = pd.read_excel("data/"+file, sheet_name=last_sheet_name, header=None)

In [25]:
def xls_to_df(filename, base_dir="data/"):
    filepath = os.path.join(base_dir, filename)

    # Check file extension and specify engine if necessary
    if filename.lower().endswith('.xls'):
        test = pd.ExcelFile(filepath, engine='xlrd')  # Use xlrd for .xls files
    else:
        test = pd.ExcelFile(filepath, engine='openpyxl')  # Use openpyxl for .xlsx files

    sheet_names = test.sheet_names
    last_sheet_name = None
    
    # Sort the sheet names if they follow the 'Sheet' naming pattern
    sheet_names = sorted([name for name in sheet_names if name.strip().lower().startswith('sheet')],
                         key=lambda name: int(re.search(r'\d+', name).group()))
    
    # Select the last sheet that matches the pattern
    if sheet_names:
        last_sheet_name = sheet_names[-1]
    
    if last_sheet_name:
        df = pd.read_excel(filepath, sheet_name=last_sheet_name, header=None)
        return df
    else:
        print(f"No matching sheets found in {filename}")
        return None

In [26]:
def clean_value(value):
    if value == 'nan':
        return ''
    if value is not None:
        value = str(value)
        value = re.sub(r"^\s*:\s*", "", value.strip())
        # value = re.sub(r"(D.*O.* No|Effectivity Date|--continued)\s*.*", "", value, flags=re.IGNORECASE).strip() 
        value = re.sub(r"(D\.?\s*O\s*\.?\s*No|Effectivity Date)\s*.*", "", value, flags=re.IGNORECASE).strip()
        value = re.sub(r"\s*-*\s*(\s*\(cont\s*\.\)|(?:\()?\s*continued\s*(?:\))|(?:\()?\s*continuation\s*(?:\))?)", "", value, flags=re.IGNORECASE).strip()
        return value
    return value

def test_function():
    pass

def extract_value(pattern, text):
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    else:
        return None

In [27]:

def find_column_headers(df, index, proximity_window=6, debug=False):
    headers = {
        'street_name_index': None,
        'vicinity_index': None,
        'classification_index': None,
        'zv_sq_m_index': None
    }
    
    for offset in range(proximity_window-3, proximity_window):
        current_index = index + offset
        if current_index >= len(df):
            break
        row = df.iloc[current_index]
        
        for col_index, cell in enumerate(row):
            cell_value = str(cell)
            if headers['street_name_index'] is None and re.search(r"(STREET NAME|SUBDIVISION|CONDOMINIUM)", cell_value, re.IGNORECASE):
                headers['street_name_index'] = col_index
            
            if headers['vicinity_index'] is None and re.search(r"V.*I.*C.*I.*N.*I.*T.*Y", cell_value, re.IGNORECASE):
                headers['vicinity_index'] = col_index
            
            if isinstance(headers['vicinity_index'], int) and cell_value == 'nan' and col_index == headers['vicinity_index'] + 1:
                headers['vicinity_index'] = [headers['vicinity_index'], col_index]
            
            if headers['classification_index'] is None and re.search(r"CLASS(?:IFICATION)?|C.*L.*A.*S.*S.*I.*F.*I.*C.*A.*T.*I.*O.*N", cell_value, re.IGNORECASE | re.DOTALL):
                headers['classification_index'] = col_index
            
            if headers['zv_sq_m_index'] is None and re.search(r"ZV.*SQ.*M|3rd\s*Rev", cell_value, re.IGNORECASE): 
                headers['zv_sq_m_index'] = col_index
        
        if all(value is not None for value in headers.values()):
            if debug:
                print(f"Headers found at index {current_index}")
                print(f"Header indices: {headers}")
            return True, headers, current_index + 1
    
    return False, None, index

In [28]:
def is_header_row(row):
    header_patterns = [
        r"(STREET NAME|SUBDIVISION|CONDOMINIUM)",
        r"V.*I.*C.*I.*N.*I.*T.*Y",
        r"CLASS(?:IFICATION)?|C.*L.*A.*S.*S.*I.*F.*I.*C.*A.*T.*I.*O.*N",
        r"ZV.*SQ.*M|3rd\s*Rev"
        
    ]
    
    found = {pattern: False for pattern in header_patterns}
    
    for cell in row:
        cell_value = str(cell)
        for pattern in header_patterns:
            if re.search(pattern, cell_value, re.IGNORECASE):
                found[pattern] = True
            
    
    return all(found.values())

In [29]:
def find_location_components(df, index, proximity_window=5, current_province=None, current_city=None, current_barangay=None, debug=False):
    last_matched_index = index
    expecting_values = False  # Flag to indicate we are expecting values in subsequent rows after combined labels
    found_any = False  # Flag to check if any location component is found

    for offset in range(proximity_window):
        current_index = index + offset
        if current_index >= len(df):
            break
        current_row = df.iloc[current_index]
        combined_current_row = ''.join(map(str, current_row.dropna())).strip()
        non_null_cells = current_row.dropna().astype(str).tolist()
        
        if debug:
            print(f"Row {current_index}: {non_null_cells}")

        # Check if this row contains the combined labels
        if not expecting_values and any(re.search(r"PROVINCE\s*/\s*CITY\s*/\s*MUNICIPALITY\s*/\s*BARANGAYS", cell, re.IGNORECASE) for cell in non_null_cells):
            expecting_values = True
            if debug:
                print(f"Combined labels found at row {current_index}")
            continue  # Move to the next row to read values

        # If we're expecting values after combined labels
        if expecting_values:
            # Iterate over cells to find values starting with ":"
            for cell in non_null_cells:
                cell = cell.strip()
                if cell.startswith(":"):
                    value = cell.lstrip(":").strip()
                    if not current_province:
                        current_province = clean_value(value)
                        found_any = True
                        if debug:
                            print(f"Province found: {current_province}")
                    elif not current_city:
                        current_city = clean_value(value)
                        found_any = True
                        if debug:
                            print(f"City/Municipality found: {current_city}")
                    elif not current_barangay:
                        current_barangay = clean_value(value)
                        found_any = True
                        if debug:
                            print(f"Barangay found: {current_barangay}")
            last_matched_index = current_index
            # If all components have values (either found now or already had values), we can return
            if (current_province and current_city and current_barangay) or offset == proximity_window - 1:
                return current_province, current_city, current_barangay, last_matched_index
            continue  # Continue to next row to find remaining components

        # Original logic for separate labels
        else:
            # Check for Province
            province = extract_value(r"Province\s*[:\s]?\s*(.*)", combined_current_row)
            if province:
                current_province = clean_value(province)
                found_any = True
                last_matched_index = current_index
                if debug:
                    print(f"Province match found in row {current_index}: {current_province}")

            # Check for City/Municipality
            city = extract_value(r"(?:City|Municipality)(?:\s*/\s*(?:City|Municipality))?\s*[:\s]?\s*(.+)", combined_current_row)
            if city:
                current_city = clean_value(city)
                found_any = True
                last_matched_index = current_index
                if debug:
                    print(f"City/Municipality match found in row {current_index}: {current_city}")

            # Check for Barangay/Zone
            barangay = extract_value(r"(?:Barangay|Zone|Barangays)(?:\s*/\s*(?:Barangay|Zone|Barangays))?\s*[:\s]?\s*(.+)", combined_current_row)
            if barangay:
                current_barangay = clean_value(barangay)
                found_any = True
                last_matched_index = current_index
                if debug:
                    print(f"Barangay/Zone match found in row {current_index}: {current_barangay}")

            # If we've found any component, we can check if we've reached the proximity window or if all components are found
            if found_any and ((current_province and current_city and current_barangay) or offset == proximity_window - 1):
                return current_province, current_city, current_barangay, last_matched_index

    return current_province, current_city, current_barangay, last_matched_index


#Code for Standard Format

Standard Format follows: 

- each tables have complete Province, Barangay, City/mun and are in succeeding rows.
                
- Table headers can be can be read with this regex pattern: 

          r"(STREET NAME|SUBDIVISION|CONDOMINIUM)",
          r"V.*I.*C.*I.*N.*I.*T.*Y",
          r"CLASS(?:IFICATION)?|C.*L.*A.*S.*S.*I.*F.*I.*C.*A.*T.*I.*O.*N",
          r"ZV.*SQ.*M|3rd\s*Rev"

In [30]:
def main(df, debug=False):
    index = 100
    count = 0
    new_df = pd.DataFrame(columns=['Province', 'City/Municipality', 'Barangay', 
                                   'Street/Subdivision', 'Vicinity', 'Classification', 'ZV/SQM'])
    
    PROXIMITY_WINDOW = 5  # Increased to accommodate different formats

    current_province = None
    current_city = None
    current_barangay = None
    header_indices = None

    # while index < len(df):
    while index < 115:
        # Attempt to find location components starting from current index
        current_province_new, current_city_new, current_barangay_new, last_matched_index = find_location_components(
            df, index, proximity_window=PROXIMITY_WINDOW, 
            current_province=current_province, current_city=current_city, current_barangay=current_barangay, debug=debug)
        
        # Update current location components with any new values
        current_province = current_province_new
        current_city = current_city_new
        current_barangay = current_barangay_new

        # Attempt to find headers starting from the last matched index
        found_headers, header_indices_new, new_index = find_column_headers(df, last_matched_index + 1, debug=debug)
        
        if found_headers:
            # Update header indices
            header_indices = header_indices_new
            index = new_index  # Move index to after headers

            # Start processing data rows
            count += 1
            if debug:
                print(f'Processing table {count}\n')

            age = 0
            MAX_AGE = 3
            col1_holder = None
            vicinity_holder = None

            while index < len(df) and age < MAX_AGE:
                row = df.iloc[index]

                # Check for new location components in the current row
                current_province_new, current_city_new, current_barangay_new, _ = find_location_components(
                    df, index, proximity_window=1, 
                    current_province=current_province, current_city=current_city, current_barangay=current_barangay, debug=debug)

                # Update current location components with any new values
                current_province = current_province_new
                current_city = current_city_new
                current_barangay = current_barangay_new

                # Check for new headers in the current row
                found_headers_in_row, header_indices_in_row, new_index_in_row = find_column_headers(df, index, debug=debug)
                if found_headers_in_row:
                    # New headers found, update header indices and reset variables
                    header_indices = header_indices_in_row
                    index = new_index_in_row
                    age = 0
                    col1_holder = None
                    vicinity_holder = None
                    if debug:
                        print(f"New headers found at index {index}. Updated header indices.")
                    continue  # Start processing new table from updated index

                # Continue processing current table
                # Extract data using the header indices
                col1 = row.iloc[header_indices['street_name_index']]
                classification = row.iloc[header_indices['classification_index']]
                zv = row.iloc[header_indices['zv_sq_m_index']]

                # Handle vicinity index (single or double column)
                vicinity = ''
                if isinstance(header_indices['vicinity_index'], int):
                    vicinity = row.iloc[header_indices['vicinity_index']]
                elif isinstance(header_indices['vicinity_index'], list):
                    vicinity1 = str(row.iloc[header_indices['vicinity_index'][0]])
                    vicinity2 = str(row.iloc[header_indices['vicinity_index'][1]])
                    if vicinity1 == 'nan':
                        vicinity = vicinity2
                    elif vicinity2 == 'nan':
                        vicinity = vicinity1
                    else:
                        vicinity = f"{vicinity1}, {vicinity2}"

                if debug:
                    print(f"Data row at index {index}: {[col1, vicinity, classification, zv]}")

                # Check if both classification and ZV/SQM are empty
                if (pd.isnull(classification) or str(classification).strip() == '') and \
                   (pd.isnull(zv) or str(zv).strip() == ''):
                    index += 1
                    age += 1
                    continue  

                # Handle missing values by carrying forward the last known values
                if pd.isnull(col1) or str(col1).strip() == '':
                    col1 = col1_holder
                else:
                    col1_holder = col1

                if pd.isnull(vicinity) or str(vicinity).strip() == '':
                    vicinity = vicinity_holder
                else:
                    vicinity_holder = vicinity

                # Append to new DataFrame
                new_df.loc[len(new_df)] = [
                    current_province, 
                    current_city, 
                    current_barangay, 
                    clean_value(col1), 
                    clean_value(vicinity), 
                    clean_value(classification), 
                    clean_value(zv)
                ]

                index += 1
                age = 0
            continue  # Proceed to next iteration of the main loop
        else:
            index += 1  # No headers found, move to the next row
    if debug:
        print(f"Total tables processed: {count}")
    return new_df


In [32]:
main(df, debug=True)

Row 0: ['REPUBLIC OF THE PHILIPPINES']
Row 1: ['DEPARTMENT OF FINANCE']
Row 2: ['MANILA']
Row 3: []
Row 4: ['DEPARTMENT ORDER NO.  034 - 2023']
Row 1: ['DEPARTMENT OF FINANCE']
Row 2: ['MANILA']
Row 3: []
Row 4: ['DEPARTMENT ORDER NO.  034 - 2023']
Row 5: ['May 24, 2023']
Row 2: ['MANILA']
Row 3: []
Row 4: ['DEPARTMENT ORDER NO.  034 - 2023']
Row 5: ['May 24, 2023']
Row 6: []
Row 3: []
Row 4: ['DEPARTMENT ORDER NO.  034 - 2023']
Row 5: ['May 24, 2023']
Row 6: []
Row 7: ['SUBJECT:         IMPLEMENTATION   OF   THE   REVISED   SCHEDULES  OF ZONAL VALUES OF REAL PROPERTIES IN THE CITY OF SURIGAO']
City/Municipality match found in row 7: OF SURIGAO
Row 4: ['DEPARTMENT ORDER NO.  034 - 2023']
Row 5: ['May 24, 2023']
Row 6: []
Row 7: ['SUBJECT:         IMPLEMENTATION   OF   THE   REVISED   SCHEDULES  OF ZONAL VALUES OF REAL PROPERTIES IN THE CITY OF SURIGAO']
City/Municipality match found in row 7: OF SURIGAO
Row 8: ['                        AND THE MUNICIPALITIES OF ALEGRIA, BACUAG, CLAVER,

KeyboardInterrupt: 

In [52]:
# Running for non-QC
excel_files = [f for f in os.listdir("data/") if os.path.isfile(os.path.join("data/", f))]

for excel in excel_files:
    print(f'Processing {excel}')
    df = xls_to_df(excel)
    processed = main(df)
    
    # Split the filename and the extension
    filename, extension = os.path.splitext(excel)
    
    if extension.lower() == '.xls':
        normalized_filename = f"{filename}.xlsx"  
    elif extension.lower() == '.xlsx':
        normalized_filename = excel  
    else:
        print(f"Unsupported file format for {excel}. Skipping...")
        continue
    
    output_path = os.path.join("Output", f"Updated_{normalized_filename}")
    processed.to_excel(output_path, index=False)

    print(f'Processed file saved as: {output_path}')

print("done w all nice")

Processing RDO No. 1 - Laoag City, Ilocos Norte.xls
Processed file saved as: Output\Updated_RDO No. 1 - Laoag City, Ilocos Norte.xlsx
Processing RDO No. 10 - Bontoc, Mt. Province 2.xls
Processed file saved as: Output\Updated_RDO No. 10 - Bontoc, Mt. Province 2.xlsx
Processing RDO No. 10 - Bontoc, Mt. Province.xls
Processed file saved as: Output\Updated_RDO No. 10 - Bontoc, Mt. Province.xlsx
Processing RDO No. 105 - Surigao City, Surigao del Norte.xls
Processed file saved as: Output\Updated_RDO No. 105 - Surigao City, Surigao del Norte.xlsx
Processing RDO No. 107 - Cotabato City, Maguindanao.xls
Processed file saved as: Output\Updated_RDO No. 107 - Cotabato City, Maguindanao.xlsx
Processing RDO No. 108 - Kidapawan City, North Cotabato.xls
Processed file saved as: Output\Updated_RDO No. 108 - Kidapawan City, North Cotabato.xlsx
Processing RDO No. 110 - General Santos City, South Cotabato.xls
Processed file saved as: Output\Updated_RDO No. 110 - General Santos City, South Cotabato.xlsx
Pr

In [53]:
output_files = [f for f in os.listdir("Output")]
for output_file in output_files:
    file = pd.read_excel(f'Output/{output_file}')
    if len(file) < 2:
        print(output_file[:-5])

Updated_RDO No. 105 - Surigao City, Surigao del Norte
Updated_RDO No. 12 - Lagawe, Ifugao
Updated_RDO No. 18 - Olongapo City, Zambales
Updated_RDO No. 60 - Lucena City, North Quezon
Updated_RDO No. 8 - Baguio City
Updated_RDO No. 80 - Mandaue City, Cebu
Updated_RDO No. 9 - La Trinidad, Benguet


In [12]:
df[117:140]

,0,1,2,3
117,"Revenue Region No. 1 - Calasiao, Pangasinan",NaN,NaN,NaN
118,"Revenue District Office No. 1 - LAOAG CITY, IL...",NaN,NaN,NaN
119,NaN,NaN,NaN,NaN
120,Province :,ILOCOS NORTE,NaN,NaN
121,Municipality :,LAOAG CITY,NaN,NaN
122,Barangay / Zone :,NO. 1 - SAN LORENZO (POBLACION),DO No.,047 - 2023
123,NaN,NaN,Effectivity Date,"Oct. 6, 2023"
124,STREET NAME / SUBDIVISION / CONDOMINIUM,VICINITY,CLASSIFI\nCATION,5TH REV\nZV / SQM
125,NaN,NaN,NaN,NaN
126,DONA MARIA G. CASTRO ST (BRILLANTES STREET),D SAMONTE ST. - PRIMO LAZARO ST(P. BURGOS ST),RR,7300
